In [1]:
import yfinance as yf
import pandas as pd
from df_preprocessing import convert_to_pct
import os

import time
import datetime

In [2]:
dva_spec = False

In [3]:
acl_18_ext_2016 = ['AAPL', 'ABB', 'ABBV', 'AEP', 'AMGN', 'AMZN', 'BA', 'BAC', 'BBL', 'BCH', 'BHP', 'BP', 'BRK-A', 'BSAC', 'BUD', 'C', 'CAT', 'CELG', 'CHL', 'CHTR', 'CMCSA', 'CODI', 'CSCO', 'CVX', 'D', 'DHR', 'DIS', 'DUK', 'EXC', 'FB', 'GD', 'GE', 'GOOG', 'HD', 'HON', 'HRG', 'HSBC', 'IEP', 'INTC', 'JNJ', 'JPM', 'KO', 'LMT', 'MA', 'MCD', 'MDT', 'MMM', 'MO', 'MRK', 'MSFT', 'NEE', 'NGG', 'NVS', 'ORCL', 'PCG', 'PCLN', 'PEP', 'PFE', 'PG', 'PICO', 'PM', 'PPL', 'PTR', 'RDS-B', 'REX', 'SLB', 'SNP', 'SNY', 'SO', 'SPLP', 'SRE', 'T', 'TM', 'TOT', 'TSM', 'UL', 'UN', 'UNH', 'UPS', 'UTX', 'V', 'VZ', 'WFC', 'WMT', 'XOM']

# stock list data

In [4]:
#daily interval
stock_ticker_dict = {}

for stock_name in acl_18_ext_2016:


    start_date = datetime.datetime.strptime('2024-05-07', '%Y-%m-%d')
    end_date = start_date + datetime.timedelta(days=6)

    num_iterations = 5  # For example, you can set the number of iterations you want

    data_frames = [] # List to store each DataFrame

    c_ticker = yf.Ticker(stock_name)

    for _ in range(num_iterations):

        # Convert dates to string format required by the history function
        start_str = start_date.strftime("%Y-%m-%d")
        end_str = end_date.strftime("%Y-%m-%d")

        
        # Fetch data
        c_data = c_ticker.history(start=start_str, end=end_str, interval='1m')
        
        # Add the DataFrame to the list
        data_frames.append(c_data)
        
        # Increment the dates by 7 days
        start_date += datetime.timedelta(days=7)
        end_date += datetime.timedelta(days=7)

        time.sleep(0.1)

    merged_data = pd.concat(data_frames)

    stock_ticker_dict[stock_name] = [c_ticker, merged_data]

    break
    

In [5]:
stock_ticker_dict['AAPL'][1].head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2024-05-07 09:30:00-04:00,183.449997,184.009995,183.449997,183.800095,4686964,0.0,0.0
2024-05-07 09:31:00-04:00,183.820007,183.970001,183.690002,183.865005,410876,0.0,0.0
2024-05-07 09:32:00-04:00,183.860001,183.970001,183.669998,183.770004,457853,0.0,0.0
2024-05-07 09:33:00-04:00,183.770004,183.910004,183.620102,183.779999,309372,0.0,0.0
2024-05-07 09:34:00-04:00,183.789993,183.800003,183.335098,183.390106,386956,0.0,0.0


In [6]:
# stock_ticker_dict['ABB'][1].empty

#check empty

temp_l = []

for stock_name in acl_18_ext_2016:
    if not stock_ticker_dict[stock_name][1].empty:
        temp_l.append(stock_name)

acl_18_ext_2016 = temp_l


KeyError: 'ABB'

In [7]:
len(acl_18_ext_2016)

85

In [8]:
for stock_name in acl_18_ext_2016:

    pct_list = ['open', 'high', 'low', 'close']
    
    #original
    stock_ticker_dict[stock_name][1].drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)
    stock_ticker_dict[stock_name][1] = stock_ticker_dict[stock_name][1].reset_index()
    stock_ticker_dict[stock_name][1] = stock_ticker_dict[stock_name][1].rename(columns=lambda x: x.lower())

    #datetime info change
    #temp_dt = pd.to_datetime(stock_ticker_dict[stock_name][1].date)
    #stock_ticker_dict[stock_name][1]['datetime'] = temp_dt.dt.strftime('%Y-%m-%d')
    

    #pct
    temp_data = stock_ticker_dict[stock_name][1].copy()
    convert_to_pct(temp_data, pct_list, drop_NA=True)
    temp_data = temp_data.reset_index().drop('index', axis=1)

    stock_ticker_dict[stock_name].append(temp_data)

    #add the target
    if dva_spec:
        stock_ticker_dict[stock_name][2]['target'] = stock_ticker_dict[stock_name][1]['close'].copy()
        pct_list.append('target')

    break
    


    

In [ ]:
# #test
# dtx = sp_500_data[sp_500_data['index_date'].isin(stock_ticker_dict['AAPL'][2]['date'])].reset_index().drop('index', axis=1)
# dtx_m = pd.concat([stock_ticker_dict['AAPL'][2], dtx], axis=1, join='outer')
# # dtx = dtx.rename(columns={'index_date':'date'})
# # dtx.date = pd.to_datetime(dtx.date)
# # stock_ticker_dict['AAPL'][2].date = pd.to_datetime(stock_ticker_dict['AAPL'][2].date)
# # dtx_m = pd.merge(stock_ticker_dict['AAPL'][2], index_match_df, on='date', how='left')

In [ ]:
# display(dtx[-10:])
# display(dtx_m)
# acl_18_ext[0]

In [ ]:

display(stock_ticker_dict[acl_18_ext_2016[0]][1])
display(stock_ticker_dict[acl_18_ext_2016[0]][2])

In [9]:
save_raw = False
save_pct = True
save_pct_index = False


#export csv
folder_raw = 'EcmP_stock_L_2016_24_dva_spec/'
folder_pct = 'hf_m/'
folder_pct_index = 'stock_us_pctindex/'

if not os.path.exists(folder_raw) and save_raw:
    os.makedirs(folder_raw)

if not os.path.exists(folder_pct) and save_pct:
    os.makedirs(folder_pct)

if not os.path.exists(folder_pct_index) and save_pct_index:
    os.makedirs(folder_pct_index)

for stock_name in acl_18_ext_2016:

    if save_raw:
        file_ori = folder_raw + stock_name+'.csv'
        stock_ticker_dict[stock_name][1].to_csv(file_ori, index=False)

    if save_pct:
        file_pct = folder_pct + stock_name+'_pct.csv'
        stock_ticker_dict[stock_name][2].to_csv(file_pct, index=False)

    if save_pct_index:
        file_pct_index = folder_pct_index + stock_name + '_pct_index.csv'
        stock_ticker_dict[stock_name][3].to_csv(file_pct_index, index=False)

KeyError: 'ABB'

1) apple 
2) Lockheed Martin 
3) Boeing 
4) AT&T
5) Netflix
6) tsmc
7) asml
8) intel
9) nvidia 
10) amd
11. HLX
12. XPOF
13. WGO
14. MLTX
15. WW
16. INSW
17. MLTX
18. msft
19. intc
20.  jpmc
21. amzn
22. tsla